In [1]:
import pandas as pd
pd.options.display.max_rows = 200

In [3]:
import azure.cosmos
from pymongo import MongoClient
import json

with open('../local.settings.json') as settings_file:
    settings = json.load(settings_file)
    MONGO_URL = settings['Values']['mongo_url']
    MONGO_USERNAME = settings['Values']['mongo_username']
    MONGO_PASSWORD = settings['Values']['mongo_password']


def insert(element):
    db = get_db_connection()
    if get(element['date']) is None:
        db.insert(element)


def get_all():
    db = get_db_connection()
    return list(db.find())


def get(date):
    db = get_db_connection()
    return db.find_one({'date': date})


def get_db_connection():
    client = MongoClient(MONGO_URL)
    db = client.xkom_bot
    db.authenticate(name=MONGO_USERNAME, password=MONGO_PASSWORD)

    return db.xkom_bot_data

In [4]:
data = pd.DataFrame(get_all())
del data['_id']

In [5]:
# Check last added values
data.sort_values(by='date', ascending=False).head(5)

,Category,New price,Original price,Product name,date
2629,Podstawki chłodzące,"49,00","99,00",Podstawka chłodząca pod laptop Targus Single F...,2020-01-15 21
2628,Dyski SSD,"199,00","249,00","Dysk SSD Plextor 512GB 2,5"" SATA M8VC",2020-01-15 18
2627,Dyski SSD,"199,00","249,00","Dysk SSD Plextor 512GB 2,5"" SATA M8VC",2020-01-15 09
2626,Myszki przewodowe,"79,00","109,00",Dream Machines DM1 Pro (5000dpi),2020-01-14 18
2624,Dyski zewnętrzne i przenośne,"189,00","249,00",Dysk zewnetrzny Silicon Power Armor A30 1TB US...,2020-01-13 22


In [6]:
data['Original price'] = data['Original price'].apply(lambda x: pd.to_numeric(str(x)[:-3].replace(" ", "")))
data['New price'] = data['New price'].apply(lambda x: pd.to_numeric(str(x)[:-3].replace(" ", "")))
data['date'] = pd.to_datetime(data['date'])

In [7]:
print(data.dtypes)
data.head(5)

Category                  object
New price                  int64
Original price             int64
Product name              object
date              datetime64[ns]
dtype: object


,Category,New price,Original price,Product name,date
0,Dyski zewnętrzne i przenośne,350,429,Dysk zewnetrzny Seagate Backup Plus Hub 4TB US...,2020-01-12 10:00:00
1,Smartfony i telefony,3499,4549,Samsung Galaxy S10 G973F Prism Black 512GB,2020-01-11 22:00:00
2,Zestawy słuchawkowe,169,249,Zestaw słuchawkowy Jabra Talk 35,2020-01-11 10:00:00
3,"Monitory LED 27"" (26,5""-28,4"")",549,699,Acer Nitro QG271BII,2020-01-10 22:00:00
4,Karty pamięci microSD,27,45,Kingston 64GB microSDHC Canvas Select Plus 100...,2020-01-10 10:00:00


In [8]:
unique_categories = data['Category'].unique()
print(unique_categories)
print(data.shape)

['Dyski zewnętrzne i przenośne' 'Smartfony i telefony'
 'Zestawy słuchawkowe' 'Monitory LED 27" (26,5"-28,4")'
 'Karty pamięci microSD' 'Torby na laptopy' 'Dyski SSD'
 'Myszki bezprzewodowe' 'Procesory AMD Ryzen 9' 'Smartwatche'
 'Obudowy dysków' 'Procesory AMD APU' 'Myszki przewodowe'
 'Głośniki przenośne' 'Słuchawki bezprzewodowe' 'Obudowy do komputera'
 'Notebooki / Laptopy 15,6"' 'Pamięci RAM DDR4' 'Procesory Intel Core i9'
 'Drukarki atramentowe' 'Czytniki ebook' 'Zestawy klawiatura i mysz'
 'Procesory Intel Core i5' 'Routery' 'Programy biurowe'
 'Sterowanie ogrzewaniem' 'Płyty główne Socket AM4'
 'Notebooki / Laptopy 14,1"' 'Procesory AMD Ryzen 7' 'Soundbary'
 'Systemy Mesh Wi-Fi' 'Monitory LED 24" (23,5"-26,4")'
 'Hulajnogi elektryczne' 'Tablety 8"' 'Wideorejestratory'
 'Kamery sportowe' 'Karty graficzne AMD' 'Tablety 10"' 'Kamery IP'
 'Procesory AMD Ryzen 5' 'Głośniki komputerowe' 'Smartwatche dla dzieci'
 'Notebooki / Laptopy 17,3"' 'Wagi' 'Smartbandy'
 'Płyty główne Socket 11

In [9]:
for category_name in ['Procesory', 'Monitory', 'Telewizory', 'Kable', 'Ładowarki', 'Stacje dokujące', 'Tablety', 'Papier', 'Gry', 'Akcesoria', 'Gaming', 'Etui', 'Laptopy', 'Słuchawk', 'Drukarki', 'Konsol', 'karty graficzne', 'klawiatury', 'kamery', 'płyty główne', 'obudowy', 'głośniki', 'myszki', 'dyski', 'karty pamięci']:
    filtered = list(filter(lambda category: category_name.lower() in category.lower(), unique_categories))
    print("{}: {}".format(category_name, len(filtered)))

Procesory: 8
Monitory: 7
Telewizory: 3
Kable: 4
Ładowarki: 3
Stacje dokujące: 3
Tablety: 5
Papier: 2
Gry: 6
Akcesoria: 6
Gaming: 2
Etui: 3
Laptopy: 8
Słuchawk: 8
Drukarki: 4
Konsol: 7
karty graficzne: 3
klawiatury: 3
kamery: 4
płyty główne: 4
obudowy: 4
głośniki: 3
myszki: 2
dyski: 5
karty pamięci: 2


In [10]:
data['Category'].value_counts()

Smartfony i telefony              379
Słuchawki przewodowe              102
Dyski zewnętrzne i przenośne       83
Dyski SSD                          82
Głośniki przenośne                 79
Słuchawki bezprzewodowe            79
Klawiatury  przewodowe             66
Monitory LED 24" (23,5"-26,4")     66
Karty pamięci microSD              64
Brak                               62
Pendrive (pamięci USB)             61
Myszki przewodowe                  60
Notebooki / Laptopy 15,6"          58
Wideorejestratory                  50
Routery                            50
Kamery IP                          49
Fotele gamingowe                   44
Monitory LED 27" (26,5"-28,4")     41
Myszki bezprzewodowe               41
Urządzenia wiel. atramentowe       40
Powerbanki                         38
Monitory LED 22" (21,5"-23,4")     31
Pamięci RAM DDR4                   31
Ładowarki do smartfonów            28
Karty graficzne NVIDIA             27
Smartwatche                        26
Plecaki na l

In [11]:
print(data['Original price'].mean(), data['Original price'].std(), data['New price'].mean(), data['New price'].std())
data['Percentage discount'] = (1 - data['New price']/data['Original price']) * 100
data['Percentage discount'] = data['Percentage discount'].apply(lambda value: int(value))
print(data['Percentage discount'].mean(), data['Percentage discount'].std())
                      

737.3342205323194 939.2973392517167 598.0346007604563 815.2329757880386
24.74334600760456 14.365756356960269
